In [3]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
#import online_nmf
import copia_nmf
import time

In [110]:
def SGD(info,n_factors,n_epochs):
    import numpy as np
    '''Learn the vectors p_u and q_i with SGD.
       data is a dataset containing all ratings + some useful info (e.g. number
       of items/users).
    '''
    
    
    # n_factors = 10  # number of factors
    alpha = .01  # learning rate
    # n_epochs = 50  # number of iteration of the SGD procedure

    users = [ t[0] for t in info]
    users = list(np.unique(np.asarray(users)))
    movies = [ t[1] for t in info ]
    movies = list(np.unique(np.asarray(movies)))
    
    #print (np.unique(users))
    #print (np.unique(movies))
    #return 0,0
    n_users = len(np.unique(np.asarray(users)))#R_df.shape[0]
    n_items = len(np.unique(np.asarray(movies)))#R_df.shape[1]

    # Randomly initialize the user and item factors.
    p = np.random.normal(0, .1, (n_users, n_factors))
    q = np.random.normal(0, .1, (n_items, n_factors))

    
    # Optimization procedure
    for _ in range(n_epochs):
        for u,i,r in info:
            u_idx = users.index(u)
            i_idx = movies.index(i)
            if u_idx > n_users:
                print("user out of bounds",u_idx)
                print(u,i,r)
                return 0,0
            if i_idx > n_items:
                print("item out of bounds",i_idx)
                print(u,i,r)
                return 0,0
            err = r - np.dot(p[u_idx], q[i_idx])
            p[u_idx] += alpha * err * q[i_idx]
            q[i_idx] += alpha * err * p[u_idx]
    #print (err)
    return np.asarray(p),np.asarray(q),users,movies

def get_uir(data):
    info = []
    for i in range(len(data['userId'])):
        u = data['userId'].iloc[i]
        m = data['movieId'].iloc[i]
        r = data['rating'].iloc[i]
        info.append((u,m,r))
    return info


# def estimate(p,q,user,item)
#     return


def split_info(info):
    from numpy.random import choice
    j = 0
    train_info = []# np.copy(info)
    test_info = []

    test_ranks = choice(range(len(info)), size=int(len(info)*0.2), replace=False)
    print("test set size", int(len(info)*0.2))

    for rank_idx in range(len(info)):
        rank = info[rank_idx]
        if rank_idx in test_ranks:
            test_info.append(rank)
        else:
            train_info.append(rank)
    
    return train_info,test_info



def training_surpriselike(n_ratings, latent_factors, n_iterations):
    import numpy as np
    import pandas as pd
    from sklearn.metrics import mean_squared_error as mse
    # from time import clock
    '''
    Read data from "ratings.csv" and use SVD or NMF to factorize the users-items
    matriz for recommendations
    '''

    def test_RMSE(info, test_info, estimates):#, users, movies):
        users = [ t[0] for t in info]
        users = list(np.unique(np.asarray(users)))
        movies = [ t[1] for t in info ]
        movies = list(np.unique(np.asarray(movies)))
        
        print(len(users))
        print(len(movies))
        print(estimates.shape)
        return 0
        MSE = 0.
        
        for score in test_info:
            if not score[0] in users:
                print("skipping user",score[0])
                continue
            if not score[1] in movies:
                print("skipping movie",score[1])
                continue
                
            u_idx = users.index(score[0])
            i_idx = movies.index(score[1])
            N_user = float(len( np.nonzero(users[u_idx]) ))

            r = score[2]
            r_est = estimates[u_idx][i_idx]
            MSE += (r-r_est)*(r-r_est)/N_user   
        return np.sqrt(MSE)


    # read data from file to a table
    data = pd.read_csv("ratings.csv")
    data = data.iloc[0 : n_ratings]
    data = data[['userId', 'movieId', 'rating']]
    info = get_uir(data)

    R_df = 0
    test_df = 0
    train_info, test_info = split_info(info)
    p, q ,users_ids, movies_ids= SGD(train_info, latent_factors, n_iterations)

    train_df = 0
    model = 0
    H = 0
    U = 0
    print("users:",p.shape)
    estimates = np.matmul(p,np.transpose(q))#np.array(info)[:,2]
    print("ratings:",estimates.shape)
#    RMSE = np.sqrt( mse(reals,estimates) )
    print("RMSE:",test_RMSE(info, test_info, estimates))
    return data, R_df, test_df, p, q



#def test_RMSE(test_info, estimates, users, movies):
#    MSE = 0.
#    for score in test_info:
#        u_idx = users.index(score[0])
#        i_idx = movies.index(score[1])
#        N_user = float(len( np.nonzero(users[u_idx])) )
#
#        r = score[2]
#        r_est = estimates[u_idx][i_idx]
#        MSE += (r-r_est)*(r-r_est)/N_user
#        
#    return np.sqrt(MSE)

In [105]:
#x = [(1,2,3),(1,3,2),(1,4,4),
#     (2,1,3),(2,3,2),
#     (3,3,3),(3,1,2),
#     (4,3,2),(4,2,1)]
#train,test = split_info(x)
#print(train)
#print(test)



test set size 1
[(1, 2, 3), (1, 3, 2), (2, 1, 3), (3, 3, 3), (4, 3, 2)]
[(1, 4, 4), (2, 3, 2), (3, 1, 2), (4, 2, 1)]


In [111]:
n_ratings = 10000
latent_factors = 5
n_iterations = 10
training_start = time.time()
#data, model, movies, R_df, test, users = online_nmf.training(n_ratings, latent_factors, n_iterations)
data, R_df, test_df, p, q = training_surpriselike(n_ratings, latent_factors, n_iterations)
training_end = time.time()
#print("Elapsed time for movie recommendation training: %.2f secs" % (training_end-training_start))
#print("users:",p.shape)
#print("movies:",q.shape)

test set size 2000
users: (91, 5)
ratings: (91, 2595)
91
2889
(91, 2595)
RMSE: 0


In [39]:
def get_RMSE():
    from numpy.random import choice
    from sklearn.metrics import mean_squared_error as mse

    N_test = int(users.shape[0]*0.2)
    print ("selecting",N_test,"users")
    RMSE = 0.
    RMSEII = 0.
    numbers = choice(users.shape[0], size=N_test)
    reals = []
    estimates = []
    for user_idx in numbers:

        real_user = R_df.iloc[user_idx]
        real_user = real_user.values.reshape(-1,real_user.shape[0])[0]
        reals.append(real_user)

        est_user = users[user_idx]
        est_user = est_user.reshape(-1,est_user.shape[0])
        est_user = np.rint(model.inverse_transform(est_user))[0]
        estimates.append(est_user)

        # # this is the same that sklearn does. Just in case
        #aux = 0.
        #for j in range(len(real_user)):
        #    aux += (real_user[j]-est_user[j])*(real_user[j]-est_user[j])
        #aux = aux/float(len(real_user))
        #RMSEII += aux

    RMSE = np.sqrt( mse(reals,estimates) )
    #RMSEII = np.sqrt( RMSEII/float(len(numbers)) )
    
    return RMSE#, RMSEII

#get_RMSE()

selecting 274 users


0.338055086531968

In [30]:
def split_set(R_df):
    import pandas as pd
    from numpy.random import choice

    def contains_duplicates(X):
        return len(np.unique(X)) != len(X)

    train_df = R_df.copy(deep=True)
#    test_df = pd.DataFrame({'userId' : [], 'movieId' : [], 'rating' : []})

#    print len(R_df.columns)
    test_movies = choice(R_df.columns, size=int(R_df.shape[1]*0.2), replace=False)

#    test_movies = choice(R_df.columns, size=1000, replace=False)
    if (contains_duplicates(test_movies)):
        print("DUPS")
        return

    print("using",len(test_movies),"movies as test")

    test_ratings = []
    test_users = []

    skipped = 0
    for tm in test_movies:
        aux = R_df[tm].nonzero()[0]
        if aux.size == 0:
            skipped += 1
            continue
        fm = aux[0] # Movie
#        test_users.append(R_df.index[fm]) # user (got from its index, just in case)
        test_users.append(R_df.index.get_loc(R_df.index[fm]))
        test_ratings.append( R_df[tm][R_df.index[fm]] ) # column = Movie, row = index of first nonzero rating for Movie
        train_df[tm].iloc[R_df.index[fm]] = 0

    # data_df has some erased entries
    # R_df is the same as before
    if skipped != 0:
        print("(",skipped," skipped)")
    test_movies = [ R_df.columns.get_loc(k) for k in test_movies]


    #test_df.reset_index(inplace=True)
    #train_df.reset_index(inplace=True)
    #test_df.drop('index',axis=1,inplace=True)
    #train_df.drop('index',axis=1,inplace=True)

    return train_df,test_movies, test_users, test_ratings
# # beware of the return values !!!! split_set(R_df)

In [31]:

def get_RMSEII(users,movies,R_df,test_users,test_movies,test_ratings,model):
    import numpy as np
    #print users[list(test_users)[10]]
    #print(users[347]) just to check
    
    U = np.asarray(users)
    M = np.transpose(movies)
    test_users = np.array(test_users)
    test_movies = np.array(test_movies)
    
    estimates = []
    reals = []
    
    RMSEII = 0.
    for idx in range(len(test_users)):
        tusr = U[test_users[idx]]
        tusr = tusr.reshape(-1,tusr.shape[0])
        #tr_usr = model.inverse_transform(tusr)

        tmov = M[test_movies[idx]]
        tmov = tmov.reshape(-1,tmov.shape[0])

#        print(tusr[0])
#        print(tmov[0])
#        print(np.dot(tmov[0],tusr[0]))

        realthing = test_ratings[idx]#R_df.iloc[test_movies[idx]].iloc[test_users[idx]]
        rat = np.dot(tmov[0],tusr[0])
        RMSEII += (rat-realthing)*(rat-realthing)
    RMSEII = np.sqrt(RMSEII/float(len(test_users)))
    return RMSEII
#train_df,test_movies, test_users, test_ratings = split_set(R_df)
#get_RMSEII(users,movies,R_df,test_users,test_movies,test_ratings,model)

In [44]:
N_tests = 10
mean = 0.
for i in range(N_tests):
    train_df,test_movies, test_users, test_ratings = split_set(R_df)
    a = get_RMSEII(users,movies,R_df,test_users,test_movies,test_ratings,model)
    #print(a)
    mean += a
print("MEAN: ",mean/float(N_tests))
print("On train set:",get_RMSE())

using 2030 movies as test
using 2030 movies as test
using 2030 movies as test
using 2030 movies as test
using 2030 movies as test
using 2030 movies as test
using 2030 movies as test
using 2030 movies as test
using 2030 movies as test
using 2030 movies as test
MEAN:  3.2735227300443306
selecting 274 users
On train set: 0.3982622532428444
